In [1]:
import lcm

In [2]:
original = lcm.EventLog("lcmlog__2015-05-17__22-34-38-022582", "r")

In [3]:
event = original.read_next_event()

In [4]:
event.timestamp

1431916478227557L

In [5]:
event.channel

'PMD_ORDERS2'

In [6]:
import drake as lcmdrake

In [7]:
len(lcmdrake.lcmt_viewer_draw._get_packed_fingerprint())

8

In [8]:
event.data[:8] == lcmdrake.lcmt_viewer_draw._get_packed_fingerprint()

False

In [9]:
import inspect

In [10]:
lcmtypes = [c[1] for c in inspect.getmembers(lcmdrake) if inspect.isclass(c[1]) and hasattr(c[1], "_get_packed_fingerprint")]
{c._get_packed_fingerprint(): c for c in lcmtypes}

{'\x0fj\xd6\xb7c/L\xa0': drake.lcmt_driving_control_cmd_t.lcmt_driving_control_cmd_t,
 '\x1d\x83m"\x031\xa6\xb0': drake.lcmt_whole_body_data.lcmt_whole_body_data,
 '5\xcd\xd7\xf9\xbd\xc1\x87o': drake.lcmt_quadrotor_input_t.lcmt_quadrotor_input_t,
 '=\x12\xab\xabc2\x89~': drake.lcmt_zmp_data.lcmt_zmp_data,
 'AO\x0b\xfe[/BD': drake.lcmt_viewer_draw.lcmt_viewer_draw,
 "B~\xd0\x18h'\x88\xee": drake.lcmt_polynomial_matrix.lcmt_polynomial_matrix,
 'C\x11\xe0\xd8\x8cAz\xa7': drake.lcmt_qp_controller_input.lcmt_qp_controller_input,
 'F\xdc\xe7\xd9\xad\xf8rd': drake.lcmt_drake_signal.lcmt_drake_signal,
 'L"\xf6M\xc1\xd5V\x85': drake.lcmt_piecewise_polynomial.lcmt_piecewise_polynomial,
 "Q%'%\xaf\x98*c": drake.lcmt_viewer_link_data.lcmt_viewer_link_data,
 '].4\xcb2W\xdb\x07': drake.lcmt_viewer_geometry_data.lcmt_viewer_geometry_data,
 'i{\x9d6\x94\xdcXJ': drake.lcmt_polynomial.lcmt_polynomial,
 'i\xf8\x95El\x94LO': drake.lcmt_joint_pd_override.lcmt_joint_pd_override,
 's d92[e\xc5': drake.lcmt_f

In [11]:
def lcmtype_classes(module_name):
    module = __import__(module_name)
    return [c[1] for c in inspect.getmembers(module) if inspect.isclass(c[1]) and hasattr(c[1], "_get_packed_fingerprint")]

def fingerprint_map(classes):
    return {c._get_packed_fingerprint(): c for c in classes}

class MessageTypeManager(object):
    def __init__(self, lcmtype_module_names):
        self.fingerprint_to_type = {}
        for name in lcmtype_module_names:
            self.fingerprint_to_type.update(fingerprint_map(lcmtype_classes(name)))
            
        self.channel_to_type = {}
        
    def get_message_type(self, event):
        fingerprint = event.data[:8]
        return self.fingerprint_to_type[fingerprint]
    
    def decode_event(self, event):
        msg_type = self.get_message_type(event)
        return msg_type.decode(event.data)
    
        

In [13]:
manager = MessageTypeManager(["bot_core", "drake", "bot_procman", "drc"])


In [21]:
out = lcm.EventLog("rewritten", "w", overwrite=True)
original = lcm.EventLog("lcmlog__2015-05-17__22-34-38-022582", "r")
for event in original:
    msg = manager.decode_event(event)
    for field in ["utime", "timestamp"]:
        if hasattr(msg, field):
            setattr(msg, field, event.timestamp)
            break
    out.write_event(event.timestamp, event.channel, msg.encode())



In [16]:
msg = manager.decode_event(event)

In [18]:
setattr(msg, "utime", 0)

In [19]:
msg.utime

0

In [53]:
fingerprint_map(lcmtype_classes("drake"))

{'\x07\xf3\xb1\xad\xde\xb6\xaag': drake.lcmt_euler_floating_joint_state_t.lcmt_euler_floating_joint_state_t,
 '\x0fj\xd6\xb7c/L\xa0': drake.lcmt_driving_control_cmd_t.lcmt_driving_control_cmd_t,
 '\x1d\x83m"\x031\xa6\xb0': drake.lcmt_whole_body_data.lcmt_whole_body_data,
 '5\xcd\xd7\xf9\xbd\xc1\x87o': drake.lcmt_quadrotor_input_t.lcmt_quadrotor_input_t,
 '=\x12\xab\xabc2\x89~': drake.lcmt_zmp_data.lcmt_zmp_data,
 'AO\x0b\xfe[/BD': drake.lcmt_viewer_draw.lcmt_viewer_draw,
 "B~\xd0\x18h'\x88\xee": drake.lcmt_polynomial_matrix.lcmt_polynomial_matrix,
 'C\x11\xe0\xd8\x8cAz\xa7': drake.lcmt_qp_controller_input.lcmt_qp_controller_input,
 'F\xdc\xe7\xd9\xad\xf8rd': drake.lcmt_drake_signal.lcmt_drake_signal,
 'L"\xf6M\xc1\xd5V\x85': drake.lcmt_piecewise_polynomial.lcmt_piecewise_polynomial,
 'Lx\xcaC\x90jI\x87': drake.lcmt_simple_car_state_t.lcmt_simple_car_state_t,
 "Q%'%\xaf\x98*c": drake.lcmt_viewer_link_data.lcmt_viewer_link_data,
 '].4\xcb2W\xdb\x07': drake.lcmt_viewer_geometry_data.lcmt_